## Process Mining

This part is very easy it consists of turnig the sequence of transactions computed 
in task $t_5$ into an .xes or .sv file in order that it can be correctly turned by 
Apromore into a process.

In [1]:
import nbimporter
from MakeTransactions_t5 import make_cal_transactions
PATH = '../../../datasets/pmdata/'

In [2]:
t = make_cal_transactions(PATH, [1,2,3,4], n_clusters=7)
for tr in t:
    tr.sort(key = lambda x: x[0] )

In [3]:
from xml.etree.ElementTree import Element, SubElement, Comment, tostring, ElementTree
from xml.dom import minidom
from datetime import *
def time_string(year=1970, month=1, day=1, hour=0, minute=0, sec=0, msec=0 ):
    return datetime(year,month,day,hour,minute,sec, msec * 1000).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [4]:
root = Element("log", {'xes.version':"2.0", 'xes.features':"nested-attributes", 'openxes.version':"2.27"})

In [5]:
def current_tree():
    print(prettify(root))
current_tree() 

<?xml version="1.0" ?>
<log xes.version="2.0" xes.features="nested-attributes" openxes.version="2.27"/>



In [6]:
SubElement(root, "extension", name="Lifecycle", prefix="lifecycle", uri="http://www.xes-standard.org/lifecycle.xesext" )

<Element 'extension' at 0x7fd01a65ea90>

In [7]:
SubElement(root, "extension", name="Organizational", prefix="org", uri="http://www.xes-standard.org/org.xesext" )
SubElement(root, "extension", name="Time", prefix="time", uri="http://www.xes-standard.org/time.xesext" )
SubElement(root, "extension", name="Concept", prefix="concept", uri="http://www.xes-standard.org/concept.xesext" )
SubElement(root, "extension", name="Cost", prefix="cost", uri="http://www.xes-standard.org/cost.xesext" )

<Element 'extension' at 0x7fd021f64db0>

In [8]:
current_tree() 

<?xml version="1.0" ?>
<log xes.version="2.0" xes.features="nested-attributes" openxes.version="2.27">
  <extension name="Lifecycle" prefix="lifecycle" uri="http://www.xes-standard.org/lifecycle.xesext"/>
  <extension name="Organizational" prefix="org" uri="http://www.xes-standard.org/org.xesext"/>
  <extension name="Time" prefix="time" uri="http://www.xes-standard.org/time.xesext"/>
  <extension name="Concept" prefix="concept" uri="http://www.xes-standard.org/concept.xesext"/>
  <extension name="Cost" prefix="cost" uri="http://www.xes-standard.org/cost.xesext"/>
</log>



In [14]:
time_string(hour=1)

'1970-01-01T01:00:00.000Z'

In [10]:
traces = [ SubElement(root,"trace") for _ in t ]

In [18]:
for i, trace in enumerate(traces):
    SubElement(trace,"string", key="concept:name", value=f'trace_{i}' )
    for item in t[i]:
        e = SubElement(trace,"event")
        SubElement(e, "date", key="time:timestamp", value= time_string(hour=item[0]) )
        SubElement(e,"string", key="concept:name", value=item[1] )
        SubElement(e,"string", key="org:group", value="partecipant" )
        SubElement(e,"string", key="lifecycle:transition", value="complete" )

In [20]:
with open('test.xes', 'w') as f:
    f.write(prettify(root))